# Bradley & Fayyad 1998 Algorithm


### Questions:

 - What is sample size? Steinley 2007 says |X|/J
 - What is SP "Starting point"?
 - Can sample sets overlap or must they be removed from the pool?
 - Why is FMS never used?

In [1]:
import imports
import numpy as np
from sklearn.cluster import KMeans
import sklearn.datasets as skdatasets
import sklearn.metrics as skmetrics 
from metrics import accuracy
from initialisations import bradleyfayyad1998 as bf

In [2]:
dataset = skdatasets.load_iris()
data = dataset.data
target = dataset.target

## Configuration/Parameters

In [3]:
K = 3
J = 10
J = 30

In [4]:
solution = bf.generate(data, K, {'J':J})
print(solution)

[[6.84047619 3.00396825 5.72142857 2.00873016]
 [5.9652381  2.80333333 4.45904762 1.46238095]
 [5.01222222 3.42833333 1.46555556 0.24222222]]


### Metrics

Before k_means_mod() implemented:
 - Adjusted Rand Index: 0.7163421126838475
 - Accuracy score: 0.8866666666666667

In [5]:
final_est =  KMeans(n_clusters=K, init=solution, n_init=1)
final_est.fit(data)

ari = skmetrics.adjusted_rand_score(target, final_est.labels_)
print("Adjusted Rand Index:", ari)

print("Accuracy score:", accuracy.score(target, final_est.labels_))

Adjusted Rand Index: 0.7302382722834697
Accuracy score: 0.8933333333333333
